In [2]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
from tqdm import tqdm

In [3]:
account_url = "https://fx34478.us-central1.gcp.snowflakecomputing.com"
organization = "ESODLJG"
account = "RU55705"
user = "DATA228PROJECT"
email = "data228.project@gmail.com"
password = "Project228"
database = 'project'
schema = "yelp_data"

In [6]:
def get_engine():
    engine = create_engine(
        'snowflake://{user}:{password}@{account}'.format(user=user,
                                                         password=password,
                                                         account=account)
                           )
    return engine

In [7]:
def get_table_name(table):
    return f'{database}.{schema}.{table}'
    
def insert_data_frame_using_insert_statement(df, table):
    print(f'dataframe length = {df.index.size}')
    db = None
    try:
        engine = get_engine()
        db = engine.connect()
        tableName = get_table_name(table)
        columnNames = list(df.columns.values)
        columns = ','.join(columnNames)
        valuePlaceHolder = ', '.join([f':{column}' for column in columnNames])
        inserted = 0

        with tqdm(total=len(df.index)) as pbar:
            batch, tempPlaceHolder, tempValues = 0, [], {}
            for i, row in enumerate(df.values):
                tempPlaceHolder.append(
                    "(" + ', '.join([f':{column}_{batch}' for column in columnNames]) + ")")
                for i, column in enumerate(columnNames):
                    tempValues[f'{column}_{batch}'] = row[i]

                batch += 1
                if batch == 16384:
                    valuePlaceHolder = ','.join(tempPlaceHolder)
                    sql = text(
                        f'insert into {tableName}({columns}) values {valuePlaceHolder}')
                    insert = db.execute(sql, tempValues)
                    tempValues = {}
                    tempPlaceHolder = []
                    inserted += batch
                    batch = 0

                pbar.update(1)
            if batch > 0:
                valuePlaceHolder = ','.join(tempPlaceHolder)
                sql = text(f'insert into  {tableName}({columns}) values {valuePlaceHolder}')
                db.execute(sql, tempValues)
                inserted += batch

    except Exception as e:
        print("there is some error")
        raise e
    finally:
        db.close()

    print(f'{inserted} - values has been inserted in the table = {table}')

In [8]:
def get_data_from_db(query):
    query_engine = get_engine()
    df = pd.read_sql_query(query, query_engine)
    return df

In [10]:
from pyspark.sql import SparkSession
import os 

import findspark
findspark.init() 

# Create a Spark session
# Specify the paths to the Snowflake connector JARs
snowflake_jdbc_jar = "/Users/hims/Downloads/snowflake-jdbc-3.14.0.jar"
spark_snowflake_jar = "/Users/hims/Downloads/spark-snowflake_2.12-2.12.0-spark_3.4.jar"

# Create a Spark session with the Snowflake connector JARs
os.environ['PYSPARK_SUBMIT_ARGS'] = ",".join([
    '--packages net.snowflake:spark-snowflake_2.12:2.12.0-spark_3.4',
    'net.snowflake:snowflake-jdbc:3.14.0 pyspark-shell'])

if "spark" in locals():
    spark.stop()
    
spark = SparkSession.builder \
    .appName("example") \
    .config("spark.jars.packages", "net.snowflake:spark-snowflake_2.12:2.12.0-spark_3.4,net.snowflake:snowflake-jdbc:3.14.0 pyspark-shell") \
    .getOrCreate()
#     .config("spark.jars", f"{snowflake_jdbc_jar},{spark_snowflake_jar}") \
#     .getOrCreate()

# Sample data
data = [("user1", "positive"),
        ("user2", "negative")]

columns = ["user_id", "sentiment"]

# Create DataFrame
df = spark.createDataFrame(data, schema=columns)

# Snowflake connection options
snowflake_options = {
    "sfURL": account_url,
    "sfDatabase": "test",
    "sfWarehouse": "COMPUTE_WH",
    "sfSchema": "test",
    "sfUser": user,
    "sfPassword": password,
    "sfTable": "test",
    "dbtable" : "test"
}
    
# Write DataFrame to Snowflake
df.write.format("snowflake") \
    .options(**snowflake_options) \
    .mode("overwrite") \
    .save("test")
# 
# # Read data back from Snowflake for verification
read_df = spark.read.format("snowflake").options(**snowflake_options).load()

# Show the result DataFrame
read_df.show(truncate=False)


+-------+---------+
|USER_ID|SENTIMENT|
+-------+---------+
|user1  |positive |
|user2  |negative |
+-------+---------+
